# Project

1. Fine tune nnUNet_v2 with Total Segmentator checkpoint
2. Predict the test set for:
    - TotalSegmentator
    - TotalSegmentator fine tuned
3. Plot results


#### 1. Fine tune
- Run plan_and_preprocess.py
- Run fine_tune.py

#### 2. Predict


In [1]:
import subprocess, os, sys
from pathlib import Path
import shutil

redo_inference = True

path = Path("/media/data/matteo.fusconi/TOTALSEGMENTATOR/nnUNet_raw/Dataset004_Femur/")
i_paths = sorted((path / "imagesTs").iterdir())
labels = sorted((path / "labelsTs").iterdir())

base_out_path = Path("/media/data/matteo.fusconi/TOTALSEGMENTATOR/evaluation")
out_dir = base_out_path / "predictions_TS"

redo_inference = True

os.environ['nnUNet_raw'] = '/media/data/matteo.fusconi/TOTALSEGMENTATOR/nnUNet_raw'
os.environ['nnUNet_preprocessed'] = '/media/data/matteo.fusconi/TOTALSEGMENTATOR/nnUNet_preprocessed'
os.environ['nnUNet_results'] = '/media/data/matteo.fusconi/TOTALSEGMENTATOR/nnUNet_results'

out_dir = Path("/media/data/matteo.fusconi/TOTALSEGMENTATOR/evaluation/predictions_TS_finetuned")


In [23]:
for pth, lab in zip(i_paths, labels):
    if redo_inference:
        subprocess.check_output(["TotalSegmentator", 
                       "-i", str(pth),
                       "-o", str(out_dir / lab.name),
                       ])
    print()
    shutil.copy(str(lab), str(out_dir/".."/"gt"/lab.name))
    for prediction in (out_dir / lab.name).iterdir():
        if prediction.name not in ["femur_left.nii.gz", "femur_right.nii.gz"]:
            prediction.unlink()

100%|██████████| 117/117 [00:03<00:00, 34.31it/s]


100%|██████████| 117/117 [00:03<00:00, 37.45it/s]


100%|██████████| 117/117 [00:02<00:00, 54.50it/s]


100%|██████████| 117/117 [00:02<00:00, 42.49it/s]


100%|██████████| 117/117 [00:02<00:00, 42.38it/s]


100%|██████████| 117/117 [00:02<00:00, 43.27it/s]


100%|██████████| 117/117 [00:02<00:00, 39.63it/s]


100%|██████████| 117/117 [00:03<00:00, 34.66it/s]


In [3]:
import SimpleITK as sitk
import numpy as np

def calculate_metrics(gtMask, predMask):
    # Convert to NumPy arrays if they aren't already
    gtMask = np.array(gtMask)
    predMask = np.array(predMask)

    # Calculate the true positives (TP), false positives (FP), and false negatives (FN)
    tp = np.sum((gtMask == 1) & (predMask == 1))
    fp = np.sum((gtMask == 0) & (predMask == 1))
    fn = np.sum((gtMask == 1) & (predMask == 0))

    # Calculate IoU
    iou = tp / (tp + fp + fn)
    dice = 2*tp / (np.sum(predMask)+np.sum(gtMask))

    return iou, dice


def calculate_3Dmetrics(gtMask, predMask):
    # Convert to NumPy arrays if they aren't already
    gtMask = np.array(gtMask)
    predMask = np.array(predMask)

    # Calculate the true positives (TP), false positives (FP), and false negatives (FN)
    tp = np.sum((gtMask == 1) & (predMask == 1))
    fp = np.sum((gtMask == 0) & (predMask == 1))
    fn = np.sum((gtMask == 1) & (predMask == 0))

    vol_gt = np.sum(gtMask)
    vol_pr = np.sum(predMask)

    # Calculate IoU
    iou = tp / (tp + fp + fn)
    dice = 2*tp / (vol_gt+vol_pr)
    volume_sym = 1 - (vol_gt - vol_pr) / (vol_gt+vol_pr)

    return iou, dice, volume_sym


In [14]:
left = sitk.ReadImage("/media/data/matteo.fusconi/TOTALSEGMENTATOR/evaluation/predictions/image_033.nii.gz/femur_left.nii.gz")
right = sitk.ReadImage("/media/data/matteo.fusconi/TOTALSEGMENTATOR/evaluation/predictions/image_033.nii.gz/femur_right.nii.gz")

In [5]:
totseg = True
out_dir = base_out_path / "predictions_TS"
gt_paths = [out_dir/".."/"gt"/lab.name for lab in labels]
totseg_paths = [out_dir / lab.name for lab in labels]

dices = []
ious = []
vol_sym = []

for gt_p, pred_p in zip(gt_paths, totseg_paths):
    gt = sitk.GetArrayFromImage(sitk.ReadImage(gt_p))
    if totseg:
        pred_left = sitk.GetArrayFromImage(sitk.ReadImage(pred_p / "femur_left.nii.gz"))
        pred_right = sitk.GetArrayFromImage(sitk.ReadImage(pred_p / "femur_right.nii.gz"))
        pred = np.clip(pred_left+pred_right, 0, 1)
    else:
        image = sitk.ReadImage(pred_p)
        pred = sitk.GetArrayFromImage(image)

    iou, dice, vol = calculate_3Dmetrics(gt, pred)


    ious.append(iou)
    dices.append(dice)
    vol_sym.append(vol)

mIoU = np.mean(ious)
mDice = np.mean(dices)
mVol = np.mean(vol_sym)

print(f"Mean IoU  Score: {mIoU} pm {np.std(ious)}")
print(f"Mean Dice Score: {mDice} pm {np.std(dices)}")
print(f"Mean Vol Symil.: {mVol} pm {np.std(vol_sym)}")


Mean IoU  Score: 0.914405371338184 pm 0.01644786887800741
Mean Dice Score: 0.9552116190565139 pm 0.009028235301327158
Mean Vol Symil.: 1.036204394958821 pm 0.013141720289099573


In [8]:
pred.shape

(300, 336, 256)

In [4]:
# SCORE 2D
totseg = True
out_dir = base_out_path / "predictions_TS"

gt_paths = [out_dir/".."/"gt"/lab.name for lab in labels]
totseg_paths = [out_dir / lab.name for lab in labels]


iou_axial, dice_axial   = [], []
iou_coronal, dice_coronal   = [], []
iou_sagittal, dice_sagittal = [], []

for gt_p, pred_p in zip(gt_paths, totseg_paths):
    gt = sitk.GetArrayFromImage(sitk.ReadImage(gt_p))
    if totseg:
        pred_left = sitk.GetArrayFromImage(sitk.ReadImage(pred_p / "femur_left.nii.gz"))
        pred_right = sitk.GetArrayFromImage(sitk.ReadImage(pred_p / "femur_right.nii.gz"))
        pred = np.clip(pred_left+pred_right, 0, 1)
    else:
        image = sitk.ReadImage(pred_p)
        pred = sitk.GetArrayFromImage(image)

    axial, sagittal, coronal = pred.shape

    # AXIAL
    for idx in range(axial):
        slice_gt = gt[idx]
        slice_pred = pred[idx]
        
        iou, dice = calculate_metrics(slice_gt, slice_pred)

        if iou is not np.nan and iou > 0.:
            iou_axial.append(iou)
            dice_axial.append(dice)

    # SAGITTAL
    for idx in range(sagittal):
        slice_gt = gt[:,idx]
        slice_pred = pred[:,idx]
        
        iou, dice = calculate_metrics(slice_gt, slice_pred)

        if iou is not np.nan and iou > 0.:
            iou_sagittal.append(iou)
            dice_sagittal.append(dice)
    
    # CORONAL
    for idx in range(coronal):
        slice_gt = gt[...,idx]
        slice_pred = pred[...,idx]
        
        iou, dice = calculate_metrics(slice_gt, slice_pred)

        if iou is not np.nan and iou > 0.:
            iou_coronal.append(iou)
            dice_coronal.append(dice)

names = ("axial", "sagittal", "coronal", "total")

mIoU = (np.mean(iou_axial), np.mean(iou_sagittal) ,np.mean(iou_coronal), np.mean(iou_axial+iou_coronal+iou_sagittal))
mDice = (np.mean(dice_axial), np.mean(dice_sagittal) ,np.mean(dice_coronal), np.mean(dice_axial+dice_coronal+dice_sagittal))

stdIoU = (np.std(iou_axial), np.std(iou_sagittal) ,np.std(iou_coronal), np.std(iou_axial+iou_coronal+iou_sagittal))
stdDice = (np.std(dice_axial), np.std(dice_sagittal) ,np.std(dice_coronal), np.std(dice_axial+dice_coronal+dice_sagittal))
for i in range(4):
    print(names[i], ":")
    print(f"Mean IoU  Score: {mIoU[i]} pm {(stdIoU[i])}")
    print(f"Mean Dice Score: {mDice[i]} pm {(stdDice[i])}")
    print()

/tmp/ipykernel_217672/1963030436.py:15: RuntimeWarning: invalid value encountered in scalar divide
  iou = tp / (tp + fp + fn)
/tmp/ipykernel_217672/1963030436.py:16: RuntimeWarning: invalid value encountered in scalar divide
  dice = 2*tp / (np.sum(predMask)+np.sum(gtMask))


axial :
Mean IoU  Score: 0.916697760118645 pm 0.05017810201634985
Mean Dice Score: 0.9556269810415451 pm 0.03553176788144167

sagittal :
Mean IoU  Score: 0.8687653755885296 pm 0.11021271292692104
Mean Dice Score: 0.9247744172514775 pm 0.08671742056664337

coronal :
Mean IoU  Score: 0.8969834947608848 pm 0.08637232296524915
Mean Dice Score: 0.9428629094781924 pm 0.06289439288313563

total :
Mean IoU  Score: 0.8988000253120708 pm 0.08239667243817296
Mean Dice Score: 0.9440825146288226 pm 0.06152809939804095



### Total Segmentator Fine-Tuned

In [3]:
import subprocess, os, sys
from pathlib import Path
import shutil
import SimpleITK as sitk

redo_inference = True

path = Path("/media/data/matteo.fusconi/TOTALSEGMENTATOR/nnUNet_raw/Dataset004_Femur/")
i_paths = sorted((path / "imagesTs").iterdir())
labels = sorted((path / "labelsTs").iterdir())

base_out_path = Path("/media/data/matteo.fusconi/TOTALSEGMENTATOR/evaluation")
out_dir = base_out_path / "predictions_TS"

In [4]:
command =  ["nnUNetv2_predict", 
            "-i", str((path / "imagesTs")),
            "-o", str(out_dir),
            "-d", "4",
            "-c", "3d_fullres",
            "-p", "totseg_nnUNetPlans",
            "-tr", "nnUNetTrainer_20epochs",
]
subprocess.check_output(command)

/media/data/matteo.fusconi/venv/totalsegmentator/lib/python3.10/site-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
100%|██████████| 12/12 [00:02<00:00,  4.23it/s]


'shutil.copy(str(lab), str(out_dir/".."/"gt"/lab.name))\n    for prediction in (out_dir / lab.name).iterdir():\n        if prediction.name not in ["femur_left.nii.gz", "femur_right.nii.gz"]:\n            prediction.unlink()'

In [6]:
totseg = False
base_out_path = Path("/media/data/matteo.fusconi/TOTALSEGMENTATOR/evaluation")

out_ts_finetuned = base_out_path / "predictions_TS_finetuned"

gt_paths = [base_out_path/"gt"/lab.name for lab in labels]
totseg_paths = [out_ts_finetuned / lab.name for lab in labels]

dices = []
ious = []
vol_sim = []

for gt_p, pred_p in zip(gt_paths, totseg_paths):
    gt = sitk.GetArrayFromImage(sitk.ReadImage(gt_p))
    if totseg:
        pred_left = sitk.GetArrayFromImage(sitk.ReadImage(pred_p / "femur_left.nii.gz"))
        pred_right = sitk.GetArrayFromImage(sitk.ReadImage(pred_p / "femur_right.nii.gz"))
        pred = np.clip(pred_left+pred_right, 0, 1)
    else:
        image = sitk.ReadImage(pred_p)
        pred = sitk.GetArrayFromImage(image)

    iou, dice, vol = calculate_3Dmetrics(gt, pred)


    ious.append(iou)
    dices.append(dice)
    vol_sim.append(vol)

mIoU = np.mean(ious)
mDice = np.mean(dices)
mVol = np.mean(vol_sim)

print(f"Mean IoU  Score: {mIoU} pm {np.std(ious)}")
print(f"Mean Dice Score: {mDice} pm {np.std(dices)}")
print(f"Mean Vol Symil.: {mVol} pm {np.std(vol_sim)}")

# print(ious, dices)

Mean IoU  Score: 0.7528799998041585 pm 0.04321675264668205
Mean Dice Score: 0.8583252946800203 pm 0.02821011702246174
Mean Vol Symil.: 0.9909744277091789 pm 0.03768022976130416


In [23]:
totseg = False
base_out_path = Path("/media/data/matteo.fusconi/TOTALSEGMENTATOR/evaluation")

out_ts_finetuned = base_out_path / "predictions_TS_finetuned"

gt_paths = [base_out_path/"gt"/lab.name for lab in labels]
totseg_paths = [out_ts_finetuned / lab.name for lab in labels]

iou_axial, dice_axial   = [], []
iou_coronal, dice_coronal   = [], []
iou_sagittal, dice_sagittal = [], []


for gt_p, pred_p in zip(gt_paths, totseg_paths):
    gt = sitk.GetArrayFromImage(sitk.ReadImage(gt_p))
    if totseg:
        pred_left = sitk.GetArrayFromImage(sitk.ReadImage(pred_p / "femur_left.nii.gz"))
        pred_right = sitk.GetArrayFromImage(sitk.ReadImage(pred_p / "femur_right.nii.gz"))
        pred = np.clip(pred_left+pred_right, 0, 1)
    else:
        image = sitk.ReadImage(pred_p)
        pred = sitk.GetArrayFromImage(image)


    axial, sagittal, coronal = pred.shape

    # AXIAL
    for idx in range(axial):
        slice_gt = gt[idx]
        slice_pred = pred[idx]
        
        iou, dice = calculate_metrics(slice_gt, slice_pred)

        if iou is not np.nan and iou > 0.:
            iou_axial.append(iou)
            dice_axial.append(dice)

    # SAGITTAL
    for idx in range(sagittal):
        slice_gt = gt[:,idx]
        slice_pred = pred[:,idx]
        
        iou, dice = calculate_metrics(slice_gt, slice_pred)

        if iou is not np.nan and iou > 0.:
            iou_sagittal.append(iou)
            dice_sagittal.append(dice)
    
    # CORONAL
    for idx in range(coronal):
        slice_gt = gt[...,idx]
        slice_pred = pred[...,idx]
        
        iou, dice = calculate_metrics(slice_gt, slice_pred)

        if iou is not np.nan and iou > 0.:
            iou_coronal.append(iou)
            dice_coronal.append(dice)

names = ("axial", "sagittal", "coronal", "total")

mIoU = (np.mean(iou_axial), np.mean(iou_sagittal) ,np.mean(iou_coronal), np.mean(iou_axial+iou_coronal+iou_sagittal))
mDice = (np.mean(dice_axial), np.mean(dice_sagittal) ,np.mean(dice_coronal), np.mean(dice_axial+dice_coronal+dice_sagittal))

stdIoU = (np.std(iou_axial), np.std(iou_sagittal) ,np.std(iou_coronal), np.std(iou_axial+iou_coronal+iou_sagittal))
stdDice = (np.std(dice_axial), np.std(dice_sagittal) ,np.std(dice_coronal), np.std(dice_axial+dice_coronal+dice_sagittal))
for i in range(4):
    print(names[i], ":")
    print(f"Mean IoU  Score: {mIoU[i]} pm {(stdIoU[i])}")
    print(f"Mean Dice Score: {mDice[i]} pm {(stdDice[i])}")
    print()

/tmp/ipykernel_202801/119295563.py:15: RuntimeWarning: invalid value encountered in scalar divide
  iou = tp / (tp + fp + fn)
/tmp/ipykernel_202801/119295563.py:16: RuntimeWarning: invalid value encountered in scalar divide
  dice = 2*tp / (np.sum(predMask)+np.sum(gtMask))


axial :
Mean IoU  Score: 0.8570864324835585 pm 0.13743258878289794
Mean Dice Score: 0.9142926867214963 pm 0.12014417472163898

sagittal :
Mean IoU  Score: 0.6468989639179642 pm 0.22073906868962614
Mean Dice Score: 0.75860564040966 pm 0.2046557064760071

coronal :
Mean IoU  Score: 0.7142828791749091 pm 0.18889004307123028
Mean Dice Score: 0.8166506044203731 pm 0.1536421474105353

total :
Mean IoU  Score: 0.7611725668008511 pm 0.19760052311053783
Mean Dice Score: 0.8459910631954837 pm 0.1668002443090423

